In [ ]:
import pyspark 
from pyspark.sql import SparkSession,Row,DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *


spark=SparkSession.builder.master("local[1]")\
        .appName("movie_local")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark-3.5.3/jars/mysql-connector-j-9.1.0.jar")\
        .config("spark.sql.shuffle.partitions", "7") \
        .config("spark.sql.warehouse.dir", "hdfs:/user/bigdata/repository/database/") \
        .config("spark.dynamicAllocation.enabled", "true") \
        .config("spark.executor.cores", "4") \
        .config("spark.dynamicAllocation.minExecutors", "1") \
        .config("spark.dynamicAllocation.maxExecutors", "5") \
        .enableHiveSupport()\
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark

25/01/16 00:15:37 WARN Utils: Your hostname, MacBook-Pro-de-Eduardo.local resolves to a loopback address: 127.0.0.1; using 192.168.3.180 instead (on interface en0)
25/01/16 00:15:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/16 00:15:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


PySpark Version :3.5.4
PySpark Version :3.5.4


In [2]:
! ls /Users/eduardoalberto/LoadFile/data_2022*.csv



/Users/eduardoalberto/LoadFile/data_20220920.csv
/Users/eduardoalberto/LoadFile/data_20220921.csv
/Users/eduardoalberto/LoadFile/data_20220922.csv
/Users/eduardoalberto/LoadFile/data_20220923.csv
/Users/eduardoalberto/LoadFile/data_20220924.csv
/Users/eduardoalberto/LoadFile/data_20220925.csv
/Users/eduardoalberto/LoadFile/data_20220926.csv


In [4]:
spark.sql("select 1").show()

+---+
|  1|
+---+
|  1|
+---+



In [5]:
df = spark.read.option("delimiter", "\t").option("header", True).csv('/Users/eduardoalberto/LoadFile/data_2022*.csv')

df.show(truncate=False)



+---------+-------------------+---------+---------+-------------------------------------+---------------------------------------+
|nconst   |primaryName        |birthYear|deathYear|primaryProfession                    |knownForTitles                         |
+---------+-------------------+---------+---------+-------------------------------------+---------------------------------------+
|nm0000001|Fred Astaire       |1899     |1987     |soundtrack,actor,miscellaneous       |tt0050419,tt0072308,tt0031983,tt0053137|
|nm0000002|Lauren Bacall      |1924     |2014     |actress,soundtrack                   |tt0037382,tt0038355,tt0117057,tt0071877|
|nm0000003|Brigitte Bardot    |1934     |\N       |actress,soundtrack,music_department  |tt0056404,tt0057345,tt0049189,tt0054452|
|nm0000004|John Belushi       |1949     |1982     |actor,soundtrack,writer              |tt0077975,tt0072562,tt0080455,tt0078723|
|nm0000005|Ingmar Bergman     |1918     |2007     |writer,director,actor                |t

In [ ]:
import pyspark 
from pyspark.sql import SparkSession,Row,DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *

def fn_ingest (df: DataFrame) -> DataFrame :

    dfs = df.filter(" deathYear <> r'\\N' and deathYear <> r'\\N'  ")\
            .withColumn('profession_array',F.explode((F.split(F.col("primaryProfession"), ","))))\
            .withColumn('knownForTitles_array',F.explode((F.split(F.col("knownForTitles"),","))))\
            .drop_duplicates(['profession_array','knownForTitles_array'])

    
    dff = dfs.filter("profession_array <> r'\\N' and knownForTitles_array <> r'\\N'")
  
    return dff
tb = fn_ingest(df)

tb.limit(50000).write.mode("overwrite").jdbc("jdbc:mysql://localhost:3306/myDbUser", "myDbUser.movie",properties={"user": "root", "password": "mysql"})

#104.466.466
#3.449.518


In [ ]:
def fn_ingest (df: DataFrame) -> DataFrame :

    dfs = (df.filter(" birthYear <> r'\\N' and deathYear <> r'\\N' ")
                .withColumn('profession_array',(F.split(F.col("primaryProfession"), ",")))
                .withColumn('knownForTitles_array',(F.split(F.col("knownForTitles"),",")))
                .select(F.col("nconst"),
                        F.col("primaryName"),
                        F.col("birthYear"),
                        F.col("deathYear"),
                        F.explode(F.col("profession_array")).alias("profession"),
                        F.explode(F.col("knownForTitles_array")).alias("Titles"))
    )
    dffull = (dfs.filter("profession <> r'\\N' and Titles <> r'\\N' ")
                 .dropDuplicates(["profession","Titles"]))
    return dffull
